In [1]:
import time
from iot_mqtt import PumpMQTT, UltraMQTT, HeatMQTT, start_broker_if_needed, stop_broker, ControllerBeacon, _best_effort_all_off
proc = start_broker_if_needed()  # only if you don’t run the Windows service

broker = "192.168.0.100"

beacon = ControllerBeacon(broker=broker, port=1883, username="pyctl-controller", password="controller", client_id="pyctl-controller",
        status_topic="pyctl/status", heartbeat_topic="pyctl/heartbeat", heartbeat_interval=5.0, keepalive=30)

beacon.start()

pumps = PumpMQTT(broker=broker, username="pyctl-controller", password="controller",
                base_topic="pumps/01", client_id="pyctl-pumps")
ultra = UltraMQTT(broker=broker, username="pyctl-controller", password="controller",
                base_topic="ultra/01", client_id="pyctl-ultra")
heat = HeatMQTT(broker=broker, username="pyctl-controller", password="controller",
                base_topic="heat/01", client_id="pyctl-heat")

pumps.ensure_connected()
ultra.ensure_connected()
heat.ensure_connected()
time.sleep(1)  # wait for connections to settle
print("Connected to broker")

[broker] Already listening on port 1883
[ctl] Connected -> 192.168.0.100:1883
[pyctl-pumps] Connecting to 192.168.0.100:1883 (attempt 1)...
[pyctl-ultra] Connecting to 192.168.0.100:1883 (attempt 1)...
[pyctl-pumps] Connected to 192.168.0.100:1883
[pyctl-heat] Connecting to 192.168.0.100:1883 (attempt 1)...
[pyctl-ultra] Connected to 192.168.0.100:1883
[pyctl-heat] Connected to 192.168.0.100:1883
Connected to broker


In [3]:
# Show live status from each node for a couple seconds
pumps.status(seconds=2.0)
time.sleep(1)
ultra.status(seconds=2.0)
time.sleep(1)
heat.status(seconds=2.0)

pumps/01/status ONLINE
pumps/01/state/1 OFF
pumps/01/state/2 OFF
pumps/01/state/3 OFF
ultra/01/status ONLINE
ultra/01/state/1 OFF
ultra/01/state/2 OFF
ultra/01/heartbeat 1


KeyboardInterrupt: 

In [6]:
# Pumps
# pumps.on(3)                 # pump 3 ON
# time.sleep(1)
# pumps.off(3)
# time.sleep(1)

# pumps.on(2)                 # pump 2 ON
# time.sleep(1)
# pumps.off(2)
# time.sleep(1)

pumps.on(3, 5000)           # pump 1 ON for 2 s (auto-off)
# pumps.on(2, 5000)           # pump 1 ON for 2 s (auto-off)
# pumps.on(1, 5000)           # pump 1 ON for 2 s (auto-off)

[pyctl-pumps] Published 'ON:5000' to pumps/01/cmd/3


In [ ]:
# # Ultrasonic
ultra.on(2)       # ultrasonic ch1 ON for 30 s
time.sleep(20)
ultra.off(2)      # ultrasonic ch1 OFF
# time.sleep(1)

# pumps.on(3, 2000)           # pump 2 ON for 5 s (auto-off)

# # ultra.on(2)       # ultrasonic ch2 ON for 30 s
# # time.sleep(30)
# # ultra.off(2)      # ultrasonic ch2 OFF
# # time.sleep(1)

# # ultra.on_for(1, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)

In [ ]:
# Heater + thermistor demo
heat.set_target(1, 42.0)         # set target to 42C (ESP retains on set/1)
heat.pid_on(1)                   # enable PID loop on ESP
# actively request a reading now:
try:
    for i in range(50):
        t = heat.get_base_temp(1, timeout_s=5.0)
        print("Temp(ch1) =", t)
        time.sleep(1)
except TimeoutError as e:
    print("Temp read timeout:", e)

heat.pid_off(1)                  # stop PID
heat.set_pwm(1, 0)               # ensure PWM is off
heat.off(1)                      # ensure relay is off

In [ ]:
set_time = 8000
pumps.on(1, set_time)           # pump 1 ON for 2 s (auto-off)
pumps.on(2, set_time)           # pump 1 ON for 2 s (auto-off)
pumps.on(3, set_time)           # pump 1 ON for 2 s (auto-off)

ultra.on(2, set_time)       # ultrasonic ch2 ON for 30 s
ultra.on(1, set_time)       # ultrasonic ch2 ON for 30 s

heat.set_target(1, 42.0)         # set target to 42C (ESP retains on set/1)
# heat.pid_on(1)                   # enable PID loop on ESP
# time.sleep(set_time/1000)
# heat.pid_off(1)                  # stop PID
# heat.set_pwm(1, 0)               # ensure PWM is off
# heat.off(1)                      # ensure relay is off

[pyctl-pumps] Published 'ON:60000' to pumps/01/cmd/1
[pyctl-pumps] Published 'ON:60000' to pumps/01/cmd/2
[pyctl-pumps] Published 'ON:60000' to pumps/01/cmd/3
[pyctl-ultra] Published 'ON:60000' to ultra/01/cmd/2
[pyctl-ultra] Published 'ON:60000' to ultra/01/cmd/1
[pyctl-heat] Published 'SET:42.0' to heat/01/cmd/1


In [ ]:
_best_effort_all_off(pumps, ultra, heat)

pumps.disconnect()
ultra.disconnect()
heat.disconnect()

beacon.stop()
stop_broker(proc)